# BackTestのチュートリアル

## 1. 実装するBotのロジック

Ukiさんの「オープニングレンジ・ブレイクアウト」をお借りして、バックテストのtutorialを作成しました（公開くさだって感謝）。

Binanceのspot/BTCBUSD用に書いていますが、PublicAPIのみ利用のためアカウントは不要で試すことができます。



<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">お待ちかねのロジックを公開。<br>(元祖)ドテン君はブレイクアウトだが通常のHLチャネルブレイクではない。その手法は「オープニングレンジ・ブレイクアウト」と呼ばれるものである。文章での説明は面倒なのでhohetoとの会議資料をそのまま添付する。ストラテジーに関する質問には応対できません。 <a href="https://t.co/LB6mdxVZZo">pic.twitter.com/LB6mdxVZZo</a></p>&mdash; UKI (@blog_uki) <a href="https://twitter.com/blog_uki/status/981768546429448192?ref_src=twsrc%5Etfw">April 5, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

![](https://pbs.twimg.com/media/DZ_xJQFUMAEOrpS?format=jpg&name=900x900)

## 2. BreakOutAgentの実装

日本語で読み解くと以下のようになります。

「シグナル点灯した次の足の始値」ではなく、シグナル点灯直後の最良値でエントリーすることに変更した以外はほぼ忠実に書き下しています。

1. `Agent.on_clock`を10分毎に呼び出すように設定する(`Agent.clock_interval`)

2. `Agent.on_clock`内で以下の処理を行う。

    1. 前処理
       
       前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
    2. Long/Short判定
      * 現在時刻から2時間足を6本取得する。`session.ohlcv`（6本目の最後の足は未確定足。10分毎に呼ばれるたびにupdateされる）
      (ohlcv[0]-ohlcv[5]へ値を格納)
      * 0-4本目の足のレンジ幅（高値ー安値）の平均値を計算`(ohlcv['high]-ohlcv['low']).mean() * K`し、Kをかけたものを'range_width'へ保存する。
      * 最新足（未確定）の始値〜高値を計算する('diff_high')
      * 最新足（未確定）の始値〜安値を計算する('diff_low')
      * 'range_width', 'diff_high', 'diff_low'の結果からLong/Short判定
    3. オーダー執行
       1. Long判定のとき
          * すでにLongポジションがある場合は何もしない。
          * Shortポジションがあった場合はドテン（最良値、倍サイズ）
          * 上記に当てはまらない場合は最良値、通常サイズでLong
       2. Short判定のとき
           * すでにShortポジションがある場合は何もしない
           * Shortポジションがあった場合はドテン（最良値、倍サイズ）
           * 上記に当てはまらない場合は最良値、通常サイズでShort

## 依存ライブラリのインストール

### 依存ライブラリ
* numpy
* pandas
* bokeh > 3

それぞれpipでインストールしてください。

Collabの場合、bokehはversion2がはいっていて、またbokeh v2に依存するpanelというライブラリ入っている状態です。panelをアンインストールしてbokehをアップデートします。





In [26]:
# 必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall panel
! pip install bokeh > 3

# https://github.com/yasstake/rusty-bot/releases/tag/release-0.2.0
# （環境にあわせて修正お願いします）
# Linux/Google Collab用
! pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0/rbot-0.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
# 必要ライブラリーのimport
import pandas as pd

# rbotは今回提供するbacktestライブラリ
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackRunner
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

## break out Agent(bot)の実装

#### 時間の扱い

時刻について、以下の基準で設計しています。
* 時刻（ある一点を指す）: UTCでエポックタイムからのマイクロ秒(μs)。時刻０はfromの時は過去全部、toのときは将来全部（≒現在時刻）
* 期間：ほとんどで秒(s)。ログのダウンロード`Market.download`だけ日(day)
  * 例：ohlcvの足の幅などは秒を利用。

#### 実装すべき必須メソッド（BaseAgentクラスをOverride)

Botは`BaseAgent`を継承し以下の３つを実装する必要があります。
* `__init()__()`    クラスのコンストラクタです。上位クラス`BaseAgent`の初期化`super().__init()__()`をわすれないように気をつけてください。
* `clock_interval()` 次の`on_clock()`が呼び出される周期を秒で返します。
* `on_clock(self, time_us, session)` 周期的に呼び出される関数です。`time_us`が現在時刻で、`session`クラスを使って`ohlcv`をとったり`place_order`したりします。

#### オプション
* `on_tick(self, time_us, session, price, side, size)` 全ログイベントを受け取ります。
* `on_update(self, time, session, result)`オーダーが約定したり失効した場合に呼ばれます。resultの中にオーダ情報が含まれています。



In [3]:
class BreakOutAgent(BaseAgent):
    """
        Agentのクラス名は任意。
        BaseAgentを継承し、clock_interval, on_clockを実装する。
    """

    def __init__(self, param_K=1.6):
            """ super().__init()__ で上位クラスの初期化を必ずすること　"""
            super().__init__()
            self.param_K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        """ on_clockが呼び出される間隔を秒で返す。今回は10分毎にする"""
        return 60 * 10
    
    def on_clock(self, time_us, session):
        """ Botのメインロジック。on_clockで設定した秒数毎に呼ばれる """
        # 前処理/ 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）        
        if session.short_order_size or session.long_order_size:
            return 
        
        ############   メインロジック  ###################### 
        ohlcv_df = session.ohlcv(60*60*2, 6)  # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:           # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv5 = ohlcv_df[:-2]       # 過去５本足（確定）
        range_width = (ohlcv5['high'] - ohlcv5['low']).mean() * self.param_K  # 価格変動レンジの平均を計算 * K

        # Long/Short判定
        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本(未確定)        
        diff_low   =   (ohlcv_latest['open'][0] - ohlcv_latest['low'][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest['open'][0] - ohlcv_latest['high'][0])  
        detect_long = range_width  < diff_high
        
        ##########  メインロジック中に利用したindicatorのロギング（あとでグラフ化するため保存）    ##############
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range_width', time_us, range_width)

        ##########　執行戦略（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）#########
        ORDER_SIZE = 0.01     # 標準オーダーサイズ(ドテンの場合はx2)
        ORDER_LIFE = 60*10    # オーダーの有効期間（60x10秒=10分)
        
        if detect_long and (not session.long_position_size): 
            if session.short_position_size: # Shortポジションがあった場合はドテン
                session.place_order( #オーダーを発行する
                            'Buy',                      # 'Buy', 'Sell'を選択
                            session.best_buy_price,     # 最後にbuyがtakeされた価格。sell側にはbest_sell_priceを提供。
                                                        # 任意の価格が設定できるがtakeになる価格でもmakeの処理・手数料で処理している。
                            ORDER_SIZE * 2,             # オーダーサイズ BTCBUSDの場合BTC建で指定。ドテンなので倍サイズでオーダー
                            ORDER_LIFE,                 # オーダーの有効期限（秒）。この秒数をこえるとExpireする。
                            'doten Long'                # あとでログで識別できるように任意の文字列が設定できる。
                )
            else:
                session.place_order('Buy', session.best_buy_price, ORDER_SIZE, ORDER_LIFE, 'Open Long')    

        if detect_short and (not session.short_position_size): # short判定のとき
            if session.long_position_size:  # Longポジションがあった場合はドテン
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE * 2, ORDER_LIFE, 'Doten Short')
            else:
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE, ORDER_LIFE, 'Open Short') 


    # 全Tick受け取りたい時は on_tick を実装する。
    #def on_tick(self, time, session, side, price, size):
    #    pass

    ## 約定イベントを受け取りたい時は on_updateを実装する。
    #def on_update(self, time, session, result):
    #    print(str(result))
    #    pass


## 3. バックテスト用データのロード

現在はBianceのSpot取引のみ対応。

`Market.open`でバイナンス（`BN`)の通貨ペアー(`BTCBUSD`)を開く。
その後、`Market.download`で過去n日分のデータをダウンロードする。

2回目以降はディフォルトではダウンロードしない。
再ダウンロードしたい場合は、`Market.download(10, True)`のように再ダウンロードフラグをTrueにする。

In [28]:
BACKTEST_PERIOD = 5             # ダウンロード＆バックテストには時間がかかるのでテスト用に少なめに設定

binance = Market.open('BN', 'BTCBUSD')  # binance marketはあとで利用するので保存しておく

Market.download(BACKTEST_PERIOD)             # BACKTEST_PERIODより最新のログデータを差分ダウンロード
#Market.download(BACKTEST_PERIOD, True)      # 再ダウンロード (１日あたり１０秒＋アルファかかります。)


Start download from 2022-12-13T00:41:32.337376
2022-12-19T00:41:32.338Z DEBUG [rbot::exchange::binance] GAP TIME: []
2022-12-19T00:41:32.338Z DEBUG [rbot::exchange::binance] GAP DAYS: []
2022-12-19T00:41:32.339Z DEBUG [rbot::exchange::binance] download rec = 0
download 0[rec]
2022-12-19T00:41:32.339Z DEBUG [rbot::exchange::binance] insert rec=0


In [29]:
# Marketオブジェクトを表示するとデータの保持期間、保存場所がわかる
# かなり巨大なデータベースになるので、不要になったら場所を確認して手で削除してください。
# 手元の計測では、70日分で15Gありました（１日あたり200Mb)
binance

start,end
1649548800063000,1671321599451000
"""2022-04-10T00:00:00.063000""","""2022-12-17T23:59:59.451000"""
days=,251


In [30]:
# DBのある場所を表示します。かなり大きなファイルになりますので不要になった場合手動で消してください。
binance.file_name

'/Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db'

## 4. バックテスト実行

注：いまのところ、Spot取引のBinance(BN)、BTCBUSDのペアーしか対応していません。通貨ペアーは切り換えれば動いちゃうかもしれません。

In [31]:
 
back_runner = BackRunner(
        'BN',           # Binance は BNと省略します。
        'BTCBUSD',      # 通貨ペアーを選択します。
        False           # 注文時に指定するサイズが通貨ペアーの右側通貨の場合True。BinanceのBTCBUSDはBTCでサイズを指定するのでFalse
)

back_runner.maker_fee_rate = 0.1 * 0.01  # maker_feeを指定（0.1%）. takerは未実装。現在は相手板にぶつける注文をしてもmaker_feeが適用される。

In [32]:
agent = BreakOutAgent()         # Agentのインスタンスを作ります（あとで利用するので変数に保存しておきます）。

# 10日前から最新までバックテストする例（最新データは２日ぐらい前のため８日間のバックテスト）。手元のM1 MacMiniで１日あたり数秒かかります。
# まれにまだエラーがでることがあります（再現条件が突き止められていない）。その場合はPythonカーネルから再起動してみてください。
back_runner.run(
    agent,                      # backtest するagentインスタンスを指定します。
    rbot.DAYS_BEFORE(BACKTEST_PERIOD),       # 開始時刻を指定します(us)。0だとDBにある最初のデータから処理。DAYS_BEFOREはN日まえのtimestampを返すユーティリティ関数です。
    0                           # 終了時刻を指定します(us). 0だとDBにある最後のデータまで処理。
)     

back_runner                     # back testの結果概要が表示されます

2022-12-19T00:41:52.532Z DEBUG [rbot::sim::back] want on tick  false
2022-12-19T00:41:52.532Z DEBUG [rbot::sim::back] want on clock true
2022-12-19T00:41:52.532Z DEBUG [rbot::sim::back] want on event false
2022-12-19T00:41:52.532Z DEBUG [rbot::sim::back] clock interval 600
2022-12-19T00:41:52.533Z DEBUG [rbot::db::sqlite] Database open path = /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db
2022-12-19T00:41:52.554Z DEBUG [rbot::db::sqlite] update_cache_df 1670935800000000 -> 1670979000000000
2022-12-19T00:41:52.554Z DEBUG [rbot::db::sqlite] update cache duration 0
2022-12-19T00:41:57.954Z DEBUG [rbot::db::sqlite] Expire cache 2022-12-13T00:50:00.000000
2022-12-19T00:41:57.954Z DEBUG [rbot::db::df] Select from 2022-12-13T00:50:00.000000 -> 1970-01-01T00:00:00.000000
2022-12-19T00:42:13.869Z DEBUG [rbot::db::df] Select from 2022-12-13T00:50:00.000000 -> 1970-01-01T00:00:00.000000
2022-12-19T00:42:13.890Z DEBUG [rbot::db::df] ohlc "2022-12-13T12:50:00.0000

実行結果の分析

実行結果は BackRunner.resultに保存されています。pd.DataFrame形式になっていますのでpandasの機能をつかって分析します。

なお、`size`とはオーダー発行したときの単位での大きさ、`volume`は反対側の通貨での大きさという意味で使い分けています。
またドテンの場合、一つのオーダーがポジションのクローズと新規のポジション作成の２つに分けられ、`sub_id`が一つインクリメントされます。

各カラムの意味は以下のとおり

* index(`create_time`)        オーダーが作られた時刻
* `update_time`	            更新時刻（ステータスが変化したときに更新）
* `1order_id`	                オーダーID（自動付与）
* `sub_id`	                オーダーSubID（ドテンの場合、オーダーCloseとOpenに分割されそれぞれIDが付与される）
* `order_side`	            'Buy'または'Sell'
* `status`	                'Open', 'Close', 'Expire'(期限切れでオーダー無効)のどれかの状態をとる
* `open_price`	            ポジションをオープンしたときの価格
* `open_size`	                                        サイズ
* `open_volume`	                                    ボリューム
* `close_price`	            ポジションをクローズしたときの価格
* `close_size`                                        サイズ
* `close_volume`                                      ボリューム
* `order_price`	            オーダで指定した価格
* `order_size`	                                    サイズ
* `order_volume`	                                    ボリューム
* `profit`	                取引による損益
* `fee`	                    取引手数料
* `total_profit`            手数料込みの損益
* `position_change`	        ポジションの変化
* `message`	                オーダー時に指定したメッセージ文字列
* `sum_profit`	            累積損益
* `position`                  現在のポジション






In [33]:
pd.options.display.max_rows=30

df = back_runner.result
df

,update_time,order_id,sub_id,order_side,status,open_price,open_size,open_volume,close_price,close_size,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-14 13:10:00+00:00,2022-12-14 13:10:00.155000+00:00,0000-0001,0,Buy,Open,17914.61,0.01,179.1461,0.00,0.00,...,17914.61,0.01,179.1461,0.0000,0.179146,-0.179146,0.01,Open Long,-0.179146,0.01
2022-12-14 19:10:00+00:00,2022-12-14 19:20:00.188000+00:00,0000-0002,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,18223.35,0.02,364.4670,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-14 19:40:00+00:00,2022-12-14 19:50:00.235000+00:00,0000-0003,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17895.43,0.02,357.9086,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-15 13:20:00+00:00,2022-12-15 13:30:00.126000+00:00,0000-0004,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17655.44,0.02,353.1088,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-15 13:50:00+00:00,2022-12-15 13:52:10.407000+00:00,0000-0005,0,Sell,Close,17914.61,0.01,179.1461,17509.66,0.01,...,17509.66,0.01,175.0966,-4.0495,0.175097,-4.224597,-0.01,Doten Short,-4.403743,0.00
2022-12-15 13:50:00+00:00,2022-12-15 13:52:10.407000+00:00,0000-0005,1,Sell,Open,17509.66,0.01,175.0966,0.00,0.00,...,17509.66,0.01,175.0966,0.0000,0.175097,-0.175097,-0.01,Doten Short,-4.578839,-0.01


### いろいろ分析

損益などの単位は、オーダー時に指定したサイズの単位の逆側になります。

例：今回はBTCBUSDの組み合わせに対し、BTCのサイズでオーダ発行しているので、BUSDでの損益。


In [34]:
# 損益結果　 
df['sum_profit'][-1]

-4.5788393000000225

In [35]:
# 利益のピーク
df["sum_profit"].max()

-0.17914610000000003

In [36]:
# 1回の取引の最大利益
df["total_profit"].max()

0.0

In [37]:
# １回の取引の最大損失
df["total_profit"].min()

-4.224596600000023

In [38]:
# 手数料なしの場合の損益
df['profit'].sum()

-4.049500000000023

In [39]:
# 手数料合計
df['fee'].sum()

0.5293393

In [40]:
# オーダー回数・Expire数の分析
df.groupby(["order_side", "status"])["status"].count()

order_side  status
Buy         Open      1
Sell        Close     1
            Expire    3
            Open      1
Name: status, dtype: int64

In [41]:
# 勝率分析
win_trade_df  = df[ (df["status"] == "Close") &  (0 < df["total_profit"])]
loss_trade_df = df[ (df["status"] == "Close") &  (df["total_profit"] < 0)]

win_trade = len(win_trade_df)
loss_trade = len(loss_trade_df)

print("買った数:", win_trade)
print("負けた数:", loss_trade)
print("勝率:", win_trade/(loss_trade+win_trade))

買った数: 0
負けた数: 1
勝率: 0.0


In [42]:
# 約定時間分析

df['exec_time'] = df['update_time'] - df.index
df[(df['status'] != 'Expire')]['exec_time'].mean()

Timedelta('0 days 00:01:26.989666666')

## 6. グラフで確認

### まずは該当期間のOHLCV（ローソク足）チャートを書く

#### OHLCVデータの取得

まず「３.バックテスト用データのダウンロード」でデータをダウンロードしておきます。

`Market.ohlcv(開始timestamp[us], 終了timestamp[us], 足の幅[sec])`
で取得できる。以下の例では10日前のタイムスタンプを返す`rbot.DAYS_BEFORE`を利用しています。

毎回Tickデータから任意幅のOHLCVを作っているのでかなり重い処理になってしまっていますが、任意の幅のOHLCVがつくれるのでこれだけ利用しても便利だと思う。

出力はpandas.DataFrameで帰ってきます。

In [44]:
# すでに notebookの最初でbinanceインスタンスは以下のコードで作成済み。ここからohlcvを取り出す。
#binance = Market.open("BN", "BTCBUSD")
#Market.download(BACKTEST_PERIOD)

ohlcv = binance.ohlcv(DAYS_BEFORE(BACKTEST_PERIOD), 0, 60*10)

2022-12-19T00:43:11.684Z DEBUG [rbot::db::sqlite] update_cache_df 1670978580000000 -> 0
2022-12-19T00:43:11.693Z DEBUG [rbot::db::sqlite] create iter 1 microsec
2022-12-19T00:43:11.693Z DEBUG [rbot::db::sqlite] load data AFTER cache df1=(21721701, 4) df2=(1, 4)
2022-12-19T00:43:11.693Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-12-17T23:59:59.451000
2022-12-19T00:43:11.736Z DEBUG [rbot::db::df] merge len 21721700
2022-12-19T00:43:11.736Z DEBUG [rbot::db::df] merge df1=(21721701, 4)  df2=(1, 4)
2022-12-19T00:43:11.744Z DEBUG [rbot::db::sqlite] cache update diff after 2022-12-14T00:43:00.000000 
2022-12-19T00:43:11.744Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-12-14T00:43:00.000000
2022-12-19T00:43:11.745Z DEBUG [rbot::db::df] ohlcv_df, from=2022-12-14T00:43:00.000000 / to=1970-01-01T00:00:00.000000
2022-12-19T00:43:11.745Z DEBUG [rbot::db::df] Select from 2022-12-14T00:43:00.000000 -> 1970-01-01T00:00:00.000000
2022-12-19T00:43:11.937Z

In [45]:
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2022-12-14 00:40:00+00:00,17807.98,17833.76,17790.39,17795.81,645.49452
2022-12-14 00:50:00+00:00,17796.00,17825.95,17788.61,17820.68,453.84454
2022-12-14 01:00:00+00:00,17820.66,17847.12,17797.16,17800.40,808.51825
2022-12-14 01:10:00+00:00,17801.20,17810.34,17788.66,17792.92,613.36785
2022-12-14 01:20:00+00:00,17792.94,17800.00,17781.82,17795.18,392.91951
...,...,...,...,...,...
2022-12-17 23:10:00+00:00,16765.20,16765.44,16736.18,16758.41,554.39818
2022-12-17 23:20:00+00:00,16758.49,16768.48,16752.18,16766.11,462.12634
2022-12-17 23:30:00+00:00,16766.14,16778.94,16755.92,16760.97,406.63174


#### OHLCVチャートの描画

`Chart`クラスを利用してチャートを描く。
引数は、`Chart(横幅、縦幅、olhcvデータ(DataFrame型))`となっています。

`Chart`インスタンスを作成し、`show()`メソッドを呼び出すことで描画されます。

描画にはbokehライブラリ`https://bokeh.org`を使っています。また`Chart`クラスはPythonの`site-package`にインストールされているはずですので、それをみながらサブクラスを作ると自由に描画がカスタマイズできると思います。（インストール先の場所の例:`lib/python3.10/site-packages/rbot/chart/`)

In [46]:
chart = Chart(900, 400, ohlcv)
chart.show()

Loading BokehJS ...

#### OHLCVにBackTestの結果を追加する。

`Chart.draw_result`でバックテストの結果を追加表示できます。

In [47]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

chart.show()

Loading BokehJS ...

### indicatorの表示

Agentの`on_clock`内で`Agent.log_indicator`を使って保存していたindicatorを`Agent.indicator()`を使って取り出すことができます。`DataFrame`形式になっているので表示します。

In [48]:
diff_low = agent.indicator('diff_low')
diff_high = agent.indicator('diff_high')
range_width = agent.indicator('range_width')
range_width

,value
timestamp,
2022-12-14 00:50:00+00:00,368.577333
2022-12-14 01:00:00+00:00,368.577333
2022-12-14 01:10:00+00:00,360.205333
2022-12-14 01:20:00+00:00,339.249333
2022-12-14 01:30:00+00:00,308.396000
...,...
2022-12-17 23:20:00+00:00,93.518667
2022-12-17 23:30:00+00:00,93.518667
2022-12-17 23:40:00+00:00,93.518667


`Chart.new_figure(新しいパネル名, height=パネルの高さ, title=タイトル)`で新しい描画画面を作ります。

作成したパネル名を指定して、`Chart.line`でデータを表示します。

`Chart.line(描画パネル名, DataFrame, x_key=データフレーム中のx軸名, y_key=データフレーム中のy軸名, color=線の色, legend_label=凡例として表示する名前)`

In [49]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

# indicatorの表示
## 新しい描画パネルを作成
chart.new_figure('indicator_panel', height=150, title='indicator')

## indicatorパネルにDataFrameを指定して折線グラフを表示
chart.line('indicator_panel', diff_low, x_key='timestamp', y_key='value', color='#ff0000', legend_label='diff_low')
chart.line('indicator_panel', diff_high, x_key='timestamp', y_key='value', color='#00ff00', legend_label='diff_high')
chart.line('indicator_panel', range_width, x_key='timestamp', y_key='value', color='#0030ff', legend_label='range_width')

chart.show()

Loading BokehJS ...

## 7. まとめ

自分で書くのは百行ぐらいで簡単にバックテストができることが確認できたと思う。

現在はBiannceのspot/BTCBUSDしか対応していませんが、使ってみてご意見お聞かせください。
Twitter(@yasstake)でもgithubでissueあげてもOK.

人気になったらDiscord立ち上げてみたいなー。



## 8. Next Step

いろいろバックテストしてみよう

#### BreakOutAgentのパラメータ変更
以下のパラメータを変えたらどうなるか確認してみよう
1. パラメータK (現在は1.6きめうち)
2. 指値（現在はbestプライス。
   * buyのときに低い価格で指すと利益は上がるがExpire率が上がる。
   * 高い価格で指すとExpire率が下がるが利益が下がる。（この場合、本来はTakerFeeが取られるが現在はMakerFeeで計算）
3. ロット（現在は0.01 BTC)の変更
4. 2時間足から他の足を試してみる。
5. Clockのタイミングを増やす・へらす（現在は10分）
6. バックテスト期間の延長（現在はBACKTEST_PERIODは5日で設定）

さらに独自のIndicatorをつくり、独自ロジックをつくってバックテストしてみよう（OHLCVをベースに判断するロジックならば問題なく作れるはず）